In [1]:
!pip install vllm==0.6.0
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 MB 61.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 16.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 190.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 95.5 MB/s eta 0:00:00:00:0100:01
  

In [2]:
import string
from vllm import LLM, SamplingParams
from tqdm import tqdm

In [3]:
import json
with open('test_data/test_data.json','r') as f:
    dataset = json.load(f)

In [4]:
llm = LLM(model='./model')

INFO 04-02 14:05:57 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='./model', speculative_config=None, tokenizer='./model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=./model, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_output_proc=True)
INFO 04-02 14:05:58 model_runner.py:915] 

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-02 14:06:00 model_runner.py:926] Loading model weights took 5.7915 GB
INFO 04-02 14:06:01 gpu_executor.py:122] # GPU blocks: 113940, # CPU blocks: 7281
INFO 04-02 14:06:08 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-02 14:06:08 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-02 14:06:20 model_runner.py:1335] Graph capturing finished in 12 secs.


In [16]:
from bs4 import BeautifulSoup
testdata =[]
for data in dataset[:200]:
    soup = BeautifulSoup(data['MDFY_BODY'],'html.parser')
    mdfy_body = soup.get_text()
    testdata.append(mdfy_body)

In [59]:
from tqdm import tqdm
test_predicts =[]
for dataset in tqdm(testdata):
    template = string.Template("<|im_start|>system\n${system_content}<|im_end|>")
    
    system_content="당신은 주어진 뉴스를 분석하는 챗봇입니다. **지시사항**:- 주어진 뉴스에 대하여 summary, advr, stk_code, sent_score 분석하고 json 형태로 출력하세요. - summary는 1~3줄 사이로 작성합니다.- advr는 해당 본문이 광고면 1 광고가 아닐경우에 0 으로 정수 1개의 값으로 출력하세요.- stk_code는 해당 본문에서 언급된 종목명을 찾고, 그 종목명의 종목 코드를 찾아 파이썬 리스트 형태로 작성하세요. - sent_score는 해당 본문이 긍정적일경우 1 부정적일경우 -1 , 긍정적이지도 부정:적이지도 않을경우 0 으로 정수 1개의 값을 출력하세요 - 본문: 이 주어지면 결과: 다음에 json 형태로 작성하세요"
    prompts = template.safe_substitute({"system_content": system_content})
    message = dataset
    prompts += f"<|im_start|>user\n{message[:1000]}<|im_end|>"
    sampling_params = SamplingParams(
        temperature=0,
        max_tokens=1024,
        stop=["<|im_end|>"],  # 문자열로 중지 토큰 지정
        repetition_penalty=1.0
    )
    outputs = llm.generate([prompts], sampling_params)
    for output in outputs:
        predict = output.outputs[0].text
    test_predicts.append(predict)



100%|██████████| 200/200 [06:18<00:00,  1.89s/it]


In [5]:
import pickle
with open('test_predicts.pkl','rb') as p:
    test_predicts = pickle.load(p)

In [12]:
import json, ast,re
predicts=[]
for predict_str in tqdm(test_predicts):
    try:
        predict = ast.literal_eval(predict_str)
        predicts.append(predict)
    except:
        predict_str = predict_str.replace('\'', '\"')
        summary_pattern = r'"summary"\s*:\s*(.*?)\s*,\s*"sktlist"'

        summary = re.findall(summary_pattern,predict_str)[0][1:-1]
        stklist_pattern = r'"sktlist"\s*:\s*(.*?)\s*,\s*"sentiment"'
        stklist = re.findall(stklist_pattern,predict_str)[0][1:-1]
        
        sentiment_pattern = r'"sentiment"\s*:\s*(.*?)\s*,\s*"advr"'
        sentiment = re.findall(sentiment_pattern,predict_str)[0][1:-1]
        
        match = re.search(r'"advr"\s*:\s*"([^"]+)"', predict_str)
        
        if match:
            advr = match.group(1)
        

        


        predicts.append({'summary':summary,
                        'stklist':stklist,
                        'sentiment':sentiment,
                        'advr':advr})

100%|██████████| 200/200 [00:00<00:00, 9327.93it/s]


In [11]:
test_predicts[157] = "{'summary': '딸기가 제철이 되어 호텔 레스토랑에서 다양한 요리로 변신하고 있습니다. 파크 하얏트 서울의 더 라운지는 신선한 딸기를 활용한 프로모션을 진행하며, 파라다이스시티는 딸기를 주재료로 한 디저트와 콜드디시를 선보입니다. 딸기는 당도가 높고 채소와 과일의 특성을,'sktlist': ['파크 하얏트 서울', '파라다이스시티', '파라다이스 호텔앤리조트', '헤럴드경제'], 'sentiment': '1\\n\\n이 뉴스 기사는 딸기의 다양한 활용과 맛을 강조하며 긍정적인 내용을 담고 있습니다. 딸기가 제철이라는 점, 다양한 요리로 변신할 수 있다는 점, 그리고 호텔에서의 프로모션 등이 긍정적인 요소로 작용합니다. 이러한 요소들이 긍정적인 감성을 유발하여 뉴스의 감성 점수를 1로 평가할 수 있습니다.', 'advr': '0'}"



In [13]:
for p in predicts:
    sentiment = p['sentiment']
    if sentiment[0] == '0':
         p['sentiment'] ='0'
    elif sentiment[0] == '1':
        p['sentiment'] ='1'
    elif sentiment[:2] == '-1':
        p['sentiment'] ='-1'
    else:
        print(p)

In [19]:
len(predicts)

200

In [18]:
summary_predict = [a['summary'] for a in predicts]
summary_true = [a['SUMMARY'] for a in dataset[:200]]

In [30]:
summary_true[180] = '''중고거래 앱으로 간병인을 구한다며 여성을 유인한 20대 남성은 피해자를 차량에 태워 가평의 한 펜션에 이틀간 감금했다. 피해자의 지인이 경찰에 신고하자 범인은 급히 도주했으나 경찰의 추격 끝에 붙잡혔고 피해 여성은 무사히 구조됐다. 경찰 조사 결과 남성은 하반신 마비 여동생이 없었으며 범행 동기를 조사하고 있다.'''



In [31]:
from rouge_score import rouge_scorer

def calculate_rouge(reference_list, candidate_list):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    
    rouge1_f, rouge2_f, rougeL_f = [], [], []
    
    for ref, cand in zip(reference_list, candidate_list):
        scores = scorer.score(ref, cand)
        rouge1_f.append(scores['rouge1'].fmeasure)
        rouge2_f.append(scores['rouge2'].fmeasure)
        rougeL_f.append(scores['rougeL'].fmeasure)
        
    avg_rouge1 = sum(rouge1_f) / len(rouge1_f)
    avg_rouge2 = sum(rouge2_f) / len(rouge2_f)
    avg_rougeL = sum(rougeL_f) / len(rougeL_f)
    
    return {
        'rouge1': avg_rouge1,
        'rouge2': avg_rouge2,
        'rougeL': avg_rougeL
    }

# Calculate ROUGE scores
rouge_scores = calculate_rouge(summary_true, summary_predict)
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': 0.41866306569247747, 'rouge2': 0.16470138195138198, 'rougeL': 0.41540816373169315}


In [33]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference_list, candidate_list):
    smoothie = SmoothingFunction().method1
    bleu_scores = []
    
    for ref, cand in zip(reference_list, candidate_list):
        ref_tokens = ref.split()
        cand_tokens = cand.split()
        score = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smoothie)
        bleu_scores.append(score)
        
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu

# Calculate BLEU score
bleu_score = calculate_bleu(summary_true, summary_predict)
print("BLEU Score:", bleu_score)


BLEU Score: 0.11493002632996721


In [35]:
%pip install bert_score
from bert_score import score

def calculate_bertscore(reference_list, candidate_list):
    P, R, F1 = score(candidate_list, reference_list, lang='ko', verbose=True)
    avg_precision = P.mean().item()
    avg_recall = R.mean().item()
    avg_f1 = F1.mean().item()
    
    return {
        'Precision': avg_precision,
        'Recall': avg_recall,
        'F1': avg_f1
    }

# Calculate BERTScore
bertscore = calculate_bertscore(summary_true, summary_predict)
print("BERTScore:", bertscore)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 157.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 192.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 180.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.27 seconds, 157.89 sentences/sec
BERTScore: {'Precision': 0.8201870918273926, 'Recall': 0.8092353940010071, 'F1': 0.8145704865455627}


In [60]:
sentiment_predict = [a['sentiment'] for a in predicts]
sentiment_true = [a['SENT_SCORE'] for a in dataset[:200]]

In [64]:
sentiment_true = [a.split('\n')[0] for a in sentiment_true]

In [66]:
sentiment_trues = []
for sent in sentiment_true:
    if '0' in sent:
        sentiment_trues.append('0')
    elif '-1' in sent:
        sentiment_trues.append('-1')
    elif '1' in sent:
        sentiment_trues.append('1')

In [70]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

# 예측값과 실제값 예시 (실제 데이터로 대체)

# true 리스트가 중첩 리스트인 경우 처리 (첫번째 리스트만 추출)
if isinstance(sentiment_trues[0], list):
    true_flat = sentiment_trues[0]
else:
    true_flat = sentiment_trues

# 정확도 계산
accuracy = accuracy_score(true_flat, sentiment_predict)
print(f"정확도 (Accuracy): {accuracy:.4f}")

# 정밀도(precision), 재현율(recall), F1 계산 (클래스별 및 평균 제공)
precision, recall, f1, support = precision_recall_fscore_support(
    true_flat, sentiment_predict, labels=['1', '0', '-1'], average=None
)

# 클래스별 결과 출력
for label, p, r, f, s in zip(['1', '0', '-1'], precision, recall, f1, support):
    print(f"\n클래스 {label} 결과:")
    print(f"정밀도(Precision): {p:.4f}")
    print(f"재현율(Recall): {r:.4f}")
    print(f"F1-Score: {f:.4f}")
    print(f"Support(샘플 개수): {s}")

# macro 평균 F1-score (모든 클래스 중요도 동일하게 가정)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    true_flat, sentiment_predict, average='macro'
)

print("\nMacro 평균 결과:")
print(f"정밀도(Precision): {precision_macro:.4f}")
print(f"재현율(Recall): {recall_macro:.4f}")
print(f"F1-Score: {f1_macro:.4f}")

# Confusion Matrix 출력
cm = confusion_matrix(true_flat, sentiment_predict, labels=['1', '0', '-1'])
print("\nConfusion Matrix:")
print(cm)

# 전체 리포트 보기 좋게 출력
print("\nClassification Report:")
print(classification_report(true_flat, sentiment_predict, labels=['1', '0', '-1']))


정확도 (Accuracy): 0.7950

클래스 1 결과:
정밀도(Precision): 0.7805
재현율(Recall): 0.8205
F1-Score: 0.8000
Support(샘플 개수): 39

클래스 0 결과:
정밀도(Precision): 0.7681
재현율(Recall): 0.6795
F1-Score: 0.7211
Support(샘플 개수): 78

클래스 -1 결과:
정밀도(Precision): 0.8222
재현율(Recall): 0.8916
F1-Score: 0.8555
Support(샘플 개수): 83

Macro 평균 결과:
정밀도(Precision): 0.7903
재현율(Recall): 0.7972
F1-Score: 0.7922

Confusion Matrix:
[[32  7  0]
 [ 9 53 16]
 [ 0  9 74]]

Classification Report:
              precision    recall  f1-score   support

           1       0.78      0.82      0.80        39
           0       0.77      0.68      0.72        78
          -1       0.82      0.89      0.86        83

    accuracy                           0.80       200
   macro avg       0.79      0.80      0.79       200
weighted avg       0.79      0.80      0.79       200



In [71]:
advr_predict = [a['advr'] for a in predicts]
advr_true = [a['ADVR_TP'] for a in dataset[:200]]

In [ ]:
# 프롬프트 템플릿 준비
template = string.Template("""<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

\n 뉴스데이터 :${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|> 
""")

for i, conv in enumerate(tqdm(test_dataset)):
        
    # 메시지 준비
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": conv['instruction']}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })
        # 샘플링 파라미터 설정
    sampling_params = SamplingParams(
        temperature=0,
        max_tokens=1024,
        stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
        repetition_penalty=1.0
    )
    
    # 생성
    outputs = llm.generate([prompt], sampling_params)
    
    # 결과 출력
    for output in outputs:

        data[i]['sLLM_output'] = output.outputs[0].text